# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [97]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [98]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)

In [99]:
r.status_code

200

In [100]:
data_json = r.json()

In [101]:
data = pd.DataFrame(data_json['items'])

In [102]:
data.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13594,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,19405.000032/2006-84,2006-07-20T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,19146375.63,0,0,26/06/2007
1,11946,Município,Vassouras,3306206,RJ,None,Arquivado por decurso de prazo,17944.001566/2011-62,2012-04-16T03:00:00Z,Operação contratual interna,Provias 2010,Instituição Financeira Nacional,Caixa Econômica Federal,Real,850000.00,0,0,18/01/2013
2,9297,Município,Nova Iguaçu,3303500,RJ,None,Regularizado,17944.000949/2007-37,2007-07-25T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,2315218.36,1,0,02/08/2007
3,12747,Município,Niterói,3303302,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001779/2006-27,2013-12-04T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,26470000.00,1,0,19/12/2013
4,13677,Município,Barra Mansa,3300407,RJ,None,Deferido,19405.000049/2005-51,2006-05-05T03:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2669010.00,0,0,15/05/2006


###  Os três status das solicitações mais frequentes na base são: Deferido, Arquivado e Regularizado.
### Quais são suas frequências? 96,  53, 26

In [103]:
# 2) Seu código aqui
data['status'].value_counts()[0:3]

Deferido        96
Arquivado       53
Regularizado    26
Name: status, dtype: int64

In [104]:
# 3) Seu código aqui
data['Ano'] = data['data_status'].apply(lambda data: data.split('/')[2])

In [105]:
data[['status', 'Ano']].head()

,status,Ano
0,Arquivado,2007
1,Arquivado por decurso de prazo,2013
2,Regularizado,2007
3,Encaminhado à PGFN com manifestação técnica fa...,2013
4,Deferido,2006


### Frequência com que cada ano aparece.

In [106]:
# 4) Seu código aqui
data['Ano'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2023    11
2022    11
2011    10
2009     9
2015     8
2017     5
2021     5
2018     4
2006     4
2016     4
2002     4
2004     1
2003     1
Name: Ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [107]:
#1) Seu código aqui
def le_api_pvl(UF, interessado):
    if interessado.lower() == 'estado':
        string_int = 'Estado'
    else:
        string_int = 'Munic%C3%ADpio'


    url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={}&tipo_interessado={}'


    r = requests.get(url.format(UF, string_int))
    djson = r.json()
    return pd.DataFrame(djson['items'])

In [108]:
uf_ba = 'BA'
interessado_ba = 'municipio'

df_ba = le_api_pvl(uf_ba, interessado_ba)
df_ba.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,27353,Município,Amargosa,2901007,BA,PVL02.002662/2017-51,Arquivado por decurso de prazo,17944.100262/2018-53,2017-12-29T13:25:41Z,Operação contratual interna,Multissetorial,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4350000.0,0,0,14/08/2018
1,64400,Município,Morro do Chapéu,2921708,BA,PVL02.000824/2023-64,Em análise,17944.101984/2023-92,2023-05-11T20:01:23Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,8000000.0,0,0,12/05/2023
2,51749,Município,América Dourada,2901155,BA,PVL02.006841/2021-43,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1790000.0,0,1,30/12/2021
3,48389,Município,Caculé,2905008,BA,PVL02.004341/2021-77,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4200000.0,0,1,16/11/2021
4,25494,Município,Guajeru,2911659,BA,00000.000000/2029-02,Deferido,17944.000837/2017-58,2017-09-28T17:43:57Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,940755.0,0,0,06/10/2017


 ### Solicitações  com status em 'Arquivado por decurso de prazo'

In [109]:
# 2) Seu código aqui
uf_mg = 'MG'
interessado_mg = 'estado'
df_mg = le_api_pvl(uf_mg, interessado_mg)

In [110]:
decurso_de_prazo = df_mg['status'] == 'Arquivado por decurso de prazo'
df_mg['status'][decurso_de_prazo].nunique()

1

### Município da Bahia com mais solicitações deferidas:

In [113]:
# 3) Seu código aqui
df_ba.loc[df_ba['status'] == 'Deferido', 'interessado'].value_counts()[:1]

Luís Eduardo Magalhães    15
Name: interessado, dtype: int64

In [112]:
# 4) Seu código aqui
df = le_api_pvl('BA', 'Estado')
df.to_csv('dados_solicitacoes_BA.csv', index=False)